In [1]:
import os
os.environ["LVMCORE_DIR"] = "/Users/droryn/prog/lvm/lvmcore"
os.environ["LVM_MASTER_DIR"] = "/Users/droryn/work/LVM/data/sas/sdsswork/lvm/sandbox/calib"
os.environ["SAS_BASE_DIR"] = "/Users/droryn/work/LVM/data/sas"
os.environ["LVM_DATA_S"] = "/Users/droryn/work/LVM/data/sas/sdsswork/data/lvm/lco"
os.environ["LVM_SPECTRO_REDUX"] = "/Users/droryn/work/LVM/data/redux"
ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")

In [2]:
import numpy as np
from astropy.table import Table
from astropy.io import fits

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import run_quickdrp as qdrp

# define environment before importing any DRP related modules

SLITMAP = Table(drp.fibermap.data)

In [17]:
from lvmdrp.functions import imageMethod as image_tasks
from lvmdrp.functions import rssMethod as rss_tasks
from lvmdrp.functions.run_quickdrp import get_master_mjd

MJD = 60404

sci_metadata = md.get_metadata(mjd=MJD, tileid=1028723).sort_values(["expnum", "camera"])

arc_lamps = {"b": "hgne", "r": "neon", "z": "neon"}

master_mjd = get_master_mjd(MJD)


spec_paths = []
for sci in sci_metadata.to_dict("records"):
    sci_camera = sci["camera"]

    # define sci paths
    sci_path = path.full("lvm_raw", camspec=sci["camera"], **sci)
    psci_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=sci["imagetyp"], **sci)
    dsci_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=sci["imagetyp"], **sci)
    xsci_path = path.full("lvm_anc", drpver=drpver, kind="x", imagetype=sci["imagetyp"], **sci)
    wsci_path = path.full("lvm_anc", drpver=drpver, kind="w", imagetype=sci["imagetyp"], **sci)
    # lamps configuration per spectrograph channel
    lamps = arc_lamps[sci["camera"][0]]
    
    # define calibration frames paths
    masters_path = os.path.join(ORIG_MASTER_DIR, f"{master_mjd}")
    if masters_path is None:
        raise ValueError("LVM_MASTER_DIR environment variable is not defined")
    mpixmask_path = os.path.join(masters_path, f"lvm-mpixmask-{sci_camera}.fits")
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{sci_camera}.fits")
    mdark_path = os.path.join(masters_path, f"lvm-mdark-{sci_camera}.fits")
    mpixflat_path = os.path.join(masters_path, f"lvm-mpixflat-{sci_camera}.fits")
    mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{sci_camera}.fits")
    mwidth_path = os.path.join(masters_path, f"lvm-mwidth-{sci_camera}.fits")
    
    # if os.path.isfile(wsci_path):
    #     print(f"skipping {wsci_path}, file already exist")
    #     continue
    
    # preprocess frame
    image_tasks.preproc_raw_frame(in_image=sci_path, out_image=psci_path, in_mask=mpixmask_path)
    
    # detrend frame
    image_tasks.detrend_frame(in_image=psci_path, out_image=dsci_path, in_bias=mbias_path, in_dark=mdark_path, in_slitmap=Table(drp.fibermap.data))
    
    # # extract 1d spectra
    # image_tasks.extract_spectra(in_image=dsci_path, out_rss=xsci_path, in_trace=mtrace_path, method="aperture", aperture=3)
    
    # # wavelength calibrate & resample
    # iwave, fwave = SPEC_CHANNELS[sci["camera"][0]]
    # rss_tasks.create_pixel_table(in_rss=xsci_path, out_rss=wsci_path, arc_wave=mwave_path, arc_fwhm=mlsf_path)
    # rss_tasks.resample_wavelength(in_rss=wsci_path, out_rss=wsci_path, method="linear", disp_pix=0.5, start_wave=iwave, end_wave=fwave, err_sim=10, parallel=0)
    
    # # apply fiberflat correction
    # rss_tasks.apply_fiberflat(in_rss=wsci_path, out_rss=wsci_path, in_flat=mflat_path)
    
    # # list paths for spectrograph combination
    # spec_paths.append(wsci_path)

[INFO]: loading/creating metadata store with parameters tileid = 1028723, mjd = 60404 and kind = 'raw'
[INFO]: found 9 frames in stores
[INFO]: number of frames after filtering 9
[INFO]: starting preprocessing of raw image 'sdR-s-b1-00015597.fits.gz'


[INFO]: using header IMAGETYP = 'object'
[INFO]: exposure time 900.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.71, 2.80756, 2.68, 2.69] (e-/ADU)
[INFO]: masked 2016 (5.81%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 1: 973.00 +/- 1.08 (ADU)
[INFO]: masked 1698 (4.90%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 2: 980.00 +/- 1.11 (ADU)
[INFO]: masked 904 (2.61%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 3: 981.00 +/- 1.26 (ADU)
[INFO]: masked 749 (2.16%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 4: 981.00 +/- 1